<a href="https://colab.research.google.com/github/kiyarashnd/promptops-project/blob/main/promptops_qc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PromptOps Quality Control Demo (Google Colab)
پیاده‌سازی عملی برای مقاله «کنترل کیفی خروجی‌های LLM مبتنی بر PromptOps».

**رویکردها:** Git-native PromptOps و PromptLayer (اختیاری)
**ابزارها:** LangChain + MLflow

## 1) نصب وابستگی‌ها

In [ ]:
!pip -q install mlflow>=2.14 pandas pyyaml jinja2 pydantic langchain langchain-openai promptlayer

## تنظیم کلیدها با Colab Secrets (امن و مناسب GitHub)

داخل Colab از پنل **🔑 Secrets** دو مقدار بساز:

- `OPENAI_API_KEY` (ضروری)
- `PROMPTLAYER_API_KEY` (اختیاری؛ فقط برای Treatment مربوط به PromptLayer)

این سلول کلیدها را از Secrets می‌خواند و به `os.environ` می‌گذارد تا هم LangChain و هم PromptLayer بتوانند استفاده کنند.


In [ ]:

from google.colab import userdata
import os

openai_key = userdata.get("OPENAI_API_KEY")
if openai_key:
    os.environ["OPENAI_API_KEY"] = openai_key

pl_key = userdata.get("PROMPTLAYER_API_KEY")
if pl_key:
    os.environ["PROMPTLAYER_API_KEY"] = pl_key

print("OPENAI_API_KEY set?", "OPENAI_API_KEY" in os.environ)
print("PROMPTLAYER_API_KEY set?", "PROMPTLAYER_API_KEY" in os.environ)


OPENAI_API_KEY set? True
PROMPTLAYER_API_KEY set? True


## 3) Dataset

In [ ]:

import pandas as pd
texts = [
"امروز صبح در محله ما برق برای دو ساعت قطع شد.",
"دیروز در مدرسه درباره اهمیت بازیافت صحبت شد.",
"یک فروشگاه آنلاین بسته را دیرتر از وعده ارسال کرد.",
"خبر علمی می‌گفت خواب کافی تمرکز را بهتر می‌کند.",
"در شبکه‌های اجتماعی شایعه‌ای بدون منبع پخش شد."
]
df = pd.DataFrame({"id": range(1, len(texts)+1), "input_text": texts})
df


,id,input_text
0,1,امروز صبح در محله ما برق برای دو ساعت قطع شد.
1,2,دیروز در مدرسه درباره اهمیت بازیافت صحبت شد.
2,3,یک فروشگاه آنلاین بسته را دیرتر از وعده ارسال ...
3,4,خبر علمی می‌گفت خواب کافی تمرکز را بهتر می‌کند.
4,5,در شبکه‌های اجتماعی شایعه‌ای بدون منبع پخش شد.


## 4) پرامپت نسخه‌ دار (Git-native)

In [ ]:

import yaml, textwrap
from pathlib import Path

base = Path("/content/promptops")
(base/"prompts_git"/"extractor").mkdir(parents=True, exist_ok=True)

yaml_content = textwrap.dedent("""
messages:
  - role: system
    content: |
      فقط یک JSON معتبر برگردان با فیلدهای:
      topic, entities, sentiment, summary_fa
  - role: user
    content: |
      متن:
      {{ input_text }}
""")
(base/"prompts_git"/"extractor"/"v1.yaml").write_text(yaml_content, encoding="utf-8")


196

## 5) QC و Prompt Manager

In [ ]:

from pydantic import BaseModel, Field
from typing import List
from jinja2 import Template
import json

class Extraction(BaseModel):
    topic: str
    entities: List[str]
    sentiment: str
    summary_fa: str

class GitPromptManager:
    def __init__(self, base_dir):
        self.base_dir = Path(base_dir)
    def get(self, name, version):
        return yaml.safe_load((self.base_dir/name/f"{version}.yaml").read_text())['messages']
    def render(self, messages, vars):
        return [{"role": m["role"], "content": Template(m["content"]).render(**vars)} for m in messages]

def qc_validate(txt):
    try:
        Extraction.model_validate(json.loads(txt))
        return True
    except Exception:
        return False


## 6) اجرای LangChain + MLflow

In [ ]:
import os
import time
import mlflow
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

def call_llm(messages):
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
    lc = []
    for m in messages:
        lc.append(SystemMessage(content=m["content"]) if m["role"] == "system"
                  else HumanMessage(content=m["content"]))
    return llm.invoke(lc).content

mlflow.set_experiment("promptops-qc-colab")
git_mgr = GitPromptManager(base / "prompts_git")

with mlflow.start_run(run_name="git_promptops"):
    okc = 0
    latencies = []

    for _, r in df.iterrows():
        sample_id = int(r["id"])
        msgs = git_mgr.render(
            git_mgr.get("extractor", "v1"),
            {"input_text": r["input_text"]}
        )

        start = time.time()
        out = call_llm(msgs)
        latency = time.time() - start
        latencies.append(latency)

        ok = qc_validate(out)
        okc += int(ok)

        mlflow.log_metric("qc_pass", int(ok), step=sample_id)
        mlflow.log_metric("latency_s", latency, step=sample_id)

    mlflow.log_metric("qc_pass_rate", okc / len(df))
    mlflow.log_metric("latency_avg_s", sum(latencies) / len(latencies))

print("Git PromptOps run done ✅")


2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/04 11:56:28 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/04 11:56:29 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/02/04 11:56:29 INFO mlflow.store.db.utils: Updating database tables
2026/02/04 11:56:29 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/04 11:56:29 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/02/04 11:56:29 INFO alembic.runtime.migration: Running upgrade  -> 451aebb31d03, add metric step
2026/02/04 11:5

Git PromptOps run done ✅


In [ ]:
import os, time
import mlflow
from promptlayer import PromptLayer

pl = PromptLayer(api_key="pl_08b110867450bf4e6a94f0e8c7237924")

with mlflow.start_run(run_name="promptlayer_registry"):
    okc = 0
    latencies = []

    label = "prod"

    for _, r in df.iterrows():
        sample_id = int(r["id"])
        input_text = r["input_text"]

        start = time.time()

        out = pl.run(
            prompt_name="extractor",
            input_variables={"input_text": input_text},
        )

        latency = time.time() - start
        latencies.append(latency)

        ok = qc_validate(out)
        okc += int(ok)

        mlflow.log_metric("qc_pass", int(ok), step=sample_id)
        mlflow.log_metric("latency_s", latency, step=sample_id)

    mlflow.log_metric("qc_pass_rate", okc / len(df))
    mlflow.log_metric("latency_avg_s", sum(latencies) / len(latencies))

print("PromptLayer run done ✅")


PromptLayer run done ✅


## مقایسه نتایج دو رویکرد (از روی MLflow)

این سلول آخرین Runهای `git_promptops` و `promptlayer_registry` را پیدا می‌کند و یک جدول مقایسه‌ای نشان می‌دهد.


In [ ]:
import pandas as pd
import mlflow

exp = mlflow.get_experiment_by_name("promptops-qc-colab")
runs = mlflow.search_runs(experiment_ids=[exp.experiment_id])

def pick_latest(df, run_name):
    sub = df[df["tags.mlflow.runName"] == run_name].copy()
    if sub.empty:
        return None
    return sub.sort_values("start_time", ascending=False).iloc[0]

git_run = pick_latest(runs, "git_promptops")
pl_run  = pick_latest(runs, "promptlayer_registry")

rows = []
if git_run is not None:
    rows.append({
        "approach": "Git PromptOps (YAML)",
        "qc_pass_rate": float(git_run.get("metrics.qc_pass_rate", float("nan"))),
        "latency_avg_s": float(git_run.get("metrics.latency_avg_s", float("nan"))),
    })

if pl_run is not None:
    rows.append({
        "approach": "PromptLayer (Registry)",
        "qc_pass_rate": float(pl_run.get("metrics.qc_pass_rate", float("nan"))),
        "latency_avg_s": float(pl_run.get("metrics.latency_avg_s", float("nan"))),
    })

pd.DataFrame(rows)


,approach,qc_pass_rate,latency_avg_s
0,Git PromptOps (YAML),1.0,1.918964
1,PromptLayer (Registry),0.0,1.233003
